In [1]:
# import libraries
import kornia
import torch

def MyCanny(input_img:torch.Tensor, st_deviation:float, gm_threshold:float) -> torch.Tensor:

    # convert image to tensor, add batch and channel dimensions
    input_img = torch.from_numpy(input_img).unsqueeze(0).unsqueeze(0).float()

    # apply gaussian-blur to images
    s_img = kornia.filters.gaussian_blur2d(input_img, (5, 5), (st_deviation, st_deviation))

    # Sobel filters 
    sobel_filter_x = torch.tensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=torch.float32)
    sobel_filter_y = torch.tensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=torch.float32)

    # add batch and channel dimensions to Sobel filters
    sobel_filter_x = sobel_filter_x.unsqueeze(0).unsqueeze(0)
    sobel_filter_y = sobel_filter_y.unsqueeze(0).unsqueeze(0)

    # convolution
    conv_x = torch.nn.functional.conv2d(s_img, sobel_filter_x, padding="valid")
    conv_y = torch.nn.functional.conv2d(s_img, sobel_filter_y, padding="valid")

    # remove bach and channel dimensions
    conv_x = conv_x.squeeze(0).squeeze(0)
    conv_y = conv_y.squeeze(0).squeeze(0)

    # calculate gradient and direction magnitube
    gradient_magnitude = torch.sqrt(conv_x**2 + conv_y**2)
    direction_magnitude = torch.atan2(conv_y ,conv_x)

    # non-maximum suppression
    nms_img = torch.zeros_like(gradient_magnitude)
    rows = gradient_magnitude.shape[0]
    cols = gradient_magnitude.shape[1]
    pi = 3.14159

    for i in range (1, rows-1):
        for j in range (1, cols-1):
            if (0 <= direction_magnitude[i,j] < (pi/8)) or (((7*pi)/8) <= direction_magnitude[i,j] < pi):
                neighbour = gradient_magnitude[i,j-1], gradient_magnitude[i,j+1]
            elif ((pi/8) <= direction_magnitude[i,j] < ((3*pi)/8)):
                neighbour = gradient_magnitude[i-1,j+1], gradient_magnitude[i+1,j-1]
            elif (((3*pi)/8) <= direction_magnitude[i,j] < ((5*pi)/8)):
                neighbour = gradient_magnitude[i-1,j], gradient_magnitude[i+1,j]
            elif (((5*pi)/8) <= direction_magnitude[i,j] < ((7*pi)/8)):
                neighbour = gradient_magnitude[i+1,j+1], gradient_magnitude[i-1,j-1]
            
            # perform non-max-suppression
            if gradient_magnitude[i, j] >= max(neighbour):
                nms_img[i, j] = gradient_magnitude[i, j]
            else:
                nms_img[i, j] = 0.0

    # apply thresholding
    output_img = nms_img > gm_threshold

    return nms_img, output_img